1. Data Cleaning
   - Rename columns
   - Drop irrelevant columns (USERID, timestamp, free text)
   - Handle missing values (drop / fill / unknown)
   - Clean Salary & Compensation (outliers, invalids)
   - Standardize currencies
   - Fix high-cardinality text (optional before EDA)

2. Data Integration
   - Merge datasets (if any)
   - Drop duplicates
   - Drop derived or repeated features

3. Data Transformation
   - Scale data (MinMax, Standard, Z-score)
   - Decimal scaling
   - Prepare for categorical encoding (after EDA)

4. Data Reduction
   - Drop low-value features
   - Reduce category levels
   - Optional: PCA / feature selection


In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
cleaned_df = pd.read_csv('/content/drive/MyDrive/Dataset/cleaned_salary_data.csv')#, encoding='utf-8'
extra_df = pd.read_csv('/content/drive/MyDrive/Dataset/xtra_info.csv')

In [10]:
for col in cleaned_df.columns:
    print(col)


USERID
Age
Industry
Job title
Salary
Compensation
Country
State
City
Overall years of experience
Years of experience in current job
Education level
Gender
Race
Salary_USD
Compensation_USD


In [11]:
for col in extra_df.columns:
    print(col)

USERID
CGPA
Salary_month_USD
Special Bonus_GBP


Make names consistent & easy to use.

In [12]:
cleaned_df = cleaned_df.rename(columns={'Job title': 'Job_title',
    'Overall years of experience': 'Overall_experience',
    'Years of experience in current job': 'Current_job_experience',
    'Education level':'Education_level'})

In [13]:
extra_df= extra_df.rename(columns={'Special Bonus_GBP':'Special_bonus_GBP'})

In [14]:
for col in cleaned_df.columns:
    print(col)

for col in extra_df.columns:
    print(col)

USERID
Age
Industry
Job_title
Salary
Compensation
Country
State
City
Overall_experience
Current_job_experience
Education_level
Gender
Race
Salary_USD
Compensation_USD
USERID
CGPA
Salary_month_USD
Special_bonus_GBP


# DATA INTEGRATION — this is where your two datasets combine

Your datasets share one perfect key: USERID

In [15]:
final_df = cleaned_df.merge(extra_df, on='USERID', how='left')


In [16]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28012 entries, 0 to 28011
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   USERID                  28012 non-null  int64  
 1   Age                     28012 non-null  float64
 2   Industry                28012 non-null  object 
 3   Job_title               28012 non-null  object 
 4   Salary                  27983 non-null  float64
 5   Compensation            28012 non-null  float64
 6   Country                 28012 non-null  object 
 7   State                   28012 non-null  object 
 8   City                    28012 non-null  object 
 9   Overall_experience      28012 non-null  float64
 10  Current_job_experience  28012 non-null  float64
 11  Education_level         28012 non-null  object 
 12  Gender                  28012 non-null  object 
 13  Race                    28012 non-null  object 
 14  Salary_USD              27983 non-null

In [17]:
final_df.describe()

,USERID,Age,Salary,Compensation,Overall_experience,Current_job_experience,Salary_USD,Compensation_USD,CGPA,Salary_month_USD,Special_bonus_GBP
count,28012.000000,28012.000000,27983.000000,28012.000000,28012.000000,28012.000000,27983.000000,28012.000000,18607.000000,19626.000000,18998.000000
mean,14072.879837,36.097440,85463.150734,3763.831037,13.011834,9.534717,83824.093875,3724.479007,2.997865,6992.471088,998.675442
std,8131.183515,8.812897,42997.857143,4438.120177,8.258725,7.297544,42513.504410,4425.481224,0.576730,3548.445475,1523.687058
min,0.000000,17.000000,1.000000,0.000000,0.500000,0.500000,0.265000,0.000000,2.000000,0.080000,0.000000
25%,7026.750000,29.500000,54000.000000,0.000000,6.000000,3.000000,52545.000000,0.000000,2.500000,4375.000000,0.000000
50%,14075.500000,39.500000,75000.000000,2000.000000,15.500000,6.000000,74000.000000,2000.000000,3.000000,6133.790000,0.000000
75%,21114.250000,39.500000,108500.000000,5000.000000,15.500000,15.500000,105000.000000,5000.000000,3.500000,8750.000000,1886.035000
max,28179.000000,69.500000,193250.000000,12500.000000,45.500000,45.500000,241562.500000,15625.000000,4.000000,20130.210000,4999.160000


In [18]:
final_df.head()

,USERID,Age,Industry,Job_title,Salary,Compensation,Country,State,City,Overall_experience,Current_job_experience,Education_level,Gender,Race,Salary_USD,Compensation_USD,CGPA,Salary_month_USD,Special_bonus_GBP
0,0,29.5,Education (Higher Education),Research and Instruction Librarian,55000.0,0.0,United States,Massachusetts,Boston,6.0,6.0,Master's degree,Woman,White,55000.0,0.0,2.75,4583.33,NaN
1,1,29.5,Computing or Tech,Change & Internal Communications Manager,54600.0,4000.0,United Kingdom,Unknown,Cambridge,9.0,6.0,College degree,Non-binary,White,68250.0,5000.0,3.90,5687.50,0.0
2,2,29.5,"Accounting, Banking & Finance",Marketing Specialist,34000.0,2000.0,US,Tennessee,Chattanooga,3.0,3.0,College degree,Woman,White,34000.0,2000.0,3.46,2833.33,0.0
3,3,29.5,Nonprofits,Program Manager,62000.0,3000.0,USA,Wisconsin,Milwaukee,9.0,6.0,College degree,Woman,White,62000.0,3000.0,3.20,5166.67,0.0
4,4,29.5,"Accounting, Banking & Finance",Accounting Manager,60000.0,7000.0,US,South Carolina,Greenville,9.0,6.0,College degree,Woman,White,60000.0,7000.0,2.31,5000.00,0.0


**Duplicate / overlapping columns**

In [19]:
#salary per month is not needed
#can be derived from Salary_month_USD = Salary_USD / 12
final_df.drop(['Salary_month_USD'], axis=1, inplace=True)

In [22]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28012 entries, 0 to 28011
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   USERID                  28012 non-null  int64  
 1   Age                     28012 non-null  float64
 2   Industry                28012 non-null  object 
 3   Job_title               28012 non-null  object 
 4   Salary                  27983 non-null  float64
 5   Compensation            28012 non-null  float64
 6   Country                 28012 non-null  object 
 7   State                   28012 non-null  object 
 8   City                    28012 non-null  object 
 9   Overall_experience      28012 non-null  float64
 10  Current_job_experience  28012 non-null  float64
 11  Education_level         28012 non-null  object 
 12  Gender                  28012 non-null  object 
 13  Race                    28012 non-null  object 
 14  Salary_USD              27983 non-null

In [20]:
final_df.isna().sum()

,0
USERID,0
Age,0
Industry,0
Job_title,0
Salary,29
Compensation,0
Country,0
State,0
City,0
Overall_experience,0


In [23]:
final_df['CGPA'] = final_df['CGPA'].fillna(final_df['CGPA'].median())

final_df['Special_bonus_GBP'] = final_df['Special_bonus_GBP'].fillna(0)



In [24]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28012 entries, 0 to 28011
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   USERID                  28012 non-null  int64  
 1   Age                     28012 non-null  float64
 2   Industry                28012 non-null  object 
 3   Job_title               28012 non-null  object 
 4   Salary                  27983 non-null  float64
 5   Compensation            28012 non-null  float64
 6   Country                 28012 non-null  object 
 7   State                   28012 non-null  object 
 8   City                    28012 non-null  object 
 9   Overall_experience      28012 non-null  float64
 10  Current_job_experience  28012 non-null  float64
 11  Education_level         28012 non-null  object 
 12  Gender                  28012 non-null  object 
 13  Race                    28012 non-null  object 
 14  Salary_USD              27983 non-null

**Drop Derivable Data**

We already have:

Salary → raw value

Currency → original currency

Salary_USD → standardized value

Since Salary_USD = Salary × FX_rate,
Salary + Currency → derivable

We can drop Salary and Currency because salary in USD is the standardized version used for analysis.

In [25]:
exchange_rate = 1.27  # example rate

final_df['Special_bonus_GBP'] = final_df['Special_bonus_GBP'] * exchange_rate

In [26]:
final_df= final_df.rename(columns={'Special_bonus_GBP':'Special_bonus_USD'})
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28012 entries, 0 to 28011
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   USERID                  28012 non-null  int64  
 1   Age                     28012 non-null  float64
 2   Industry                28012 non-null  object 
 3   Job_title               28012 non-null  object 
 4   Salary                  27983 non-null  float64
 5   Compensation            28012 non-null  float64
 6   Country                 28012 non-null  object 
 7   State                   28012 non-null  object 
 8   City                    28012 non-null  object 
 9   Overall_experience      28012 non-null  float64
 10  Current_job_experience  28012 non-null  float64
 11  Education_level         28012 non-null  object 
 12  Gender                  28012 non-null  object 
 13  Race                    28012 non-null  object 
 14  Salary_USD              27983 non-null

In [27]:
final_df.to_csv("/content/drive/MyDrive/Dataset/merged_salary_data.csv", index=False)

# DATA TRANSFORMATION

Changing the values of your columns so they become more useful, cleaner, standardized, or prepared for analysis / ML.

**Noise**

**Aggregation — Summarizing Data**
Data cube construction

In [28]:
#aggregate salary-related columns to create summary metrics.
final_df['Total_Compensation_USD'] = (
    final_df['Salary'] +
    final_df['Compensation'] +
    (final_df['Special_bonus_USD'])  # convert GBP→USD
)

In [29]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28012 entries, 0 to 28011
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   USERID                  28012 non-null  int64  
 1   Age                     28012 non-null  float64
 2   Industry                28012 non-null  object 
 3   Job_title               28012 non-null  object 
 4   Salary                  27983 non-null  float64
 5   Compensation            28012 non-null  float64
 6   Country                 28012 non-null  object 
 7   State                   28012 non-null  object 
 8   City                    28012 non-null  object 
 9   Overall_experience      28012 non-null  float64
 10  Current_job_experience  28012 non-null  float64
 11  Education_level         28012 non-null  object 
 12  Gender                  28012 non-null  object 
 13  Race                    28012 non-null  object 
 14  Salary_USD              27983 non-null

In [33]:
final_df.groupby('Country')['Salary'].mean()


,Salary
Country,
New Zealand,86000.000000
U.S.,42500.000000
US,86000.000000
United States,69000.000000
"$2,175.84/year is deducted for benefits",61700.000000
...,...
united states of america,56000.000000
us,91344.473684
usa,89178.893048


In [34]:
final_df['Country'].unique()

array(['United States', 'United Kingdom', 'US', 'USA', 'Canada',
       'United Kingdom ', 'usa', 'UK', 'Scotland ', 'U.S.',
       'United States ', 'The Netherlands', 'Australia ', 'Spain', 'us',
       'Usa', 'England', 'finland', 'United States of America', 'France',
       'United states', 'Scotland', 'USA ', 'United states ', 'Germany',
       'UK ', 'united states', 'Ireland', 'Australia', 'Uk',
       'United States of America ', 'U.S. ', 'canada', 'Canada ', 'U.S>',
       'ISA', 'Great Britain ', 'US ', 'United State', 'U.S.A', 'Denmark',
       'U.S.A.', 'America', 'Netherlands', 'netherlands', 'England ',
       'united states of america', 'Ireland ', 'Switzerland',
       'Netherlands ', 'Bermuda', 'Us', 'The United States',
       'United State of America', 'Germany ', 'Mexico ', 'United Stated',
       'South Africa ', 'Belgium', 'Northern Ireland', 'u.s.',
       'South Africa', 'UNITED STATES', 'united States', 'Sweden',
       'Hong Kong', 'Kuwait', 'Sri lanka', 'Cont

In [35]:
us_variants = [
    "united states", "united state", "united statess", "united states of america",
    "united state of america", "united states is america", "us", "u s",
    "u s a", "u s a ", "usa", "u s ", "u s a.", "america", "usaa", "usat", "u s "
]

uk_variants = [
    "united kingdom", "uk", "u k", "england", "scotland", "wales", "great britain",
    "britain", "northern ireland", "england uk", "england gb"
]

netherlands_variants = ["netherlands", "the netherlands", "nederland", "nl"]

australia_variants = ["australia", "australian"]

canada_variants = ["canada", "can", "canada ottawa ontario", "canad", "canada and usa"]


In [36]:
def clean_country(x):

    # UNITED STATES
    if x in us_variants:
        return "United States"

    # UNITED KINGDOM
    if x in uk_variants:
        return "United Kingdom"

    # CANADA
    if x in canada_variants:
        return "Canada"

    # AUSTRALIA
    if x in australia_variants:
        return "Australia"

    # NETHERLANDS
    if x in netherlands_variants:
        return "Netherlands"

    # INDIA
    if x in ["india", "ibdia"]:
        return "India"

    # PAKISTAN
    if x in ["pakistan", "pakistan ", "company in germany i work from pakistan"]:
        return "Pakistan"

    # SINGAPORE
    if x in ["singapore", "singapore "]:
        return "Singapore"

    # MALAYSIA
    if x in ["malaysia", "malaysia "]:
        return "Malaysia"

    # PHILIPPINES
    if x in ["philippines", "remote philippines"]:
        return "Philippines"

    # GERMANY
    if x in ["germany", "germany "]:
        return "Germany"

    # FRANCE
    if x in ["france", "france "]:
        return "France"

    # UKRAINE
    if x in ["ukraine", "ukraine "]:
        return "Ukraine"

    # UAE
    if x in ["united arab emirates", "uae"]:
        return "United Arab Emirates"

    # NEW ZEALAND
    if x in ["new zealand", "aotearoa new zealand"]:
        return "New Zealand"

    ### ADD MORE AS NEEDED ###

    # DEFAULT: unknown → Other
    return "Other"

final_df["Country_Clean"] = final_df["Country"].apply(clean_country)


In [37]:
final_df['Country_Clean'].unique()

array(['Other', 'United States', 'Canada', 'Netherlands', 'Germany',
       'United Kingdom', 'Australia', 'Singapore', 'New Zealand',
       'France', 'Pakistan', 'India', 'Philippines'], dtype=object)

In [38]:
final_df['Country'].unique()

array(['United States', 'United Kingdom', 'US', 'USA', 'Canada',
       'United Kingdom ', 'usa', 'UK', 'Scotland ', 'U.S.',
       'United States ', 'The Netherlands', 'Australia ', 'Spain', 'us',
       'Usa', 'England', 'finland', 'United States of America', 'France',
       'United states', 'Scotland', 'USA ', 'United states ', 'Germany',
       'UK ', 'united states', 'Ireland', 'Australia', 'Uk',
       'United States of America ', 'U.S. ', 'canada', 'Canada ', 'U.S>',
       'ISA', 'Great Britain ', 'US ', 'United State', 'U.S.A', 'Denmark',
       'U.S.A.', 'America', 'Netherlands', 'netherlands', 'England ',
       'united states of america', 'Ireland ', 'Switzerland',
       'Netherlands ', 'Bermuda', 'Us', 'The United States',
       'United State of America', 'Germany ', 'Mexico ', 'United Stated',
       'South Africa ', 'Belgium', 'Northern Ireland', 'u.s.',
       'South Africa', 'UNITED STATES', 'united States', 'Sweden',
       'Hong Kong', 'Kuwait', 'Sri lanka', 'Cont

# Generalization

✔ Many categories
✔ Many textual columns
✔ Many numeric columns with too many unique values

Age is already generalized — it is not raw age, but abstracted age groups.

**Generalizing Education Levels**

In [39]:
final_df['Education_level'].unique()

array(["Master's degree", 'College degree', 'PhD', 'Some college',
       'High School', 'Professional degree (MD, JD, etc.)'], dtype=object)

In [40]:
edu_map = {
    "High School": "Secondary",
    "Some college": "Undergraduate",
    "College degree": "Undergraduate",
    "Master's degree": "Graduate",
    "Professional degree (MD, JD, etc.)": "Professional",
    "PhD": "Doctorate"
}

final_df["Edu_Level"] = final_df["Education_level"].map(edu_map)


In [41]:
final_df['Edu_Level'].unique()

array(['Graduate', 'Undergraduate', 'Doctorate', 'Secondary',
       'Professional'], dtype=object)

In [44]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28012 entries, 0 to 28011
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   USERID                  28012 non-null  int64  
 1   Age                     28012 non-null  float64
 2   Industry                28012 non-null  object 
 3   Job_title               28012 non-null  object 
 4   Salary                  27983 non-null  float64
 5   Compensation            28012 non-null  float64
 6   Country                 28012 non-null  object 
 7   State                   28012 non-null  object 
 8   City                    28012 non-null  object 
 9   Overall_experience      28012 non-null  float64
 10  Current_job_experience  28012 non-null  float64
 11  Education_level         28012 non-null  object 
 12  Gender                  28012 non-null  object 
 13  Race                    28012 non-null  object 
 14  Salary_USD              27983 non-null

**Generalizing Job_title Levels**

In [45]:
len(final_df['Job_title'].unique())


14344

In [46]:
def map_job_family(title):
    if pd.isna(title):
        return "Other"

    t = title.lower()

    # ===== DATA & AI =====
    if any(k in t for k in [
        'data', 'ml', 'machine learning', 'ai', 'artificial intelligence',
        'analytics', 'data scientist', 'statistic'
    ]):
        return "Data / AI"

    # ===== SOFTWARE & ENGINEERING =====
    if any(k in t for k in [
        'engineer', 'developer', 'devops', 'swe', 'software', 'programmer',
        'full stack', 'backend', 'frontend', 'cloud'
    ]):
        return "Software Engineering"

    # ===== IT SUPPORT / SYSTEMS =====
    if any(k in t for k in [
        'it support', 'helpdesk', 'tech support', 'system admin', 'sysadmin',
        'network', 'infrastructure', 'desktop support'
    ]):
        return "IT Systems / Support"

    # ===== PRODUCT / PROJECT / SCRUM =====
    if any(k in t for k in [
        'product', 'project', 'program manager', 'scrum', 'agile', 'pmo'
    ]):
        return "Product / Project"

    # ===== DESIGN / CREATIVE =====
    if any(k in t for k in [
        'designer', 'ux', 'ui', 'graphic', 'creative', 'illustrator'
    ]):
        return "Design / UX"

    # ===== SALES / BUSINESS DEV =====
    if any(k in t for k in [
        'sales', 'account manager', 'business development', 'bdm', 'inside sales'
    ]):
        return "Sales / Business"

    # ===== MARKETING =====
    if any(k in t for k in [
        'marketing', 'seo', 'content', 'social media', 'brand'
    ]):
        return "Marketing"

    # ===== FINANCE / ACCOUNTING =====
    if any(k in t for k in [
        'finance', 'accountant', 'fp&a', 'auditor', 'tax', 'bookkeeper', 'controller'
    ]):
        return "Finance / Accounting"

    # ===== HR / RECRUITING =====
    if any(k in t for k in [
        'hr', 'human resources', 'recruiter', 'talent', 'people operations'
    ]):
        return "HR / Recruiting"

    # ===== HEALTHCARE =====
    if any(k in t for k in [
        'nurse', 'doctor', 'physician', 'medical', 'clinical', 'healthcare'
    ]):
        return "Healthcare"

    # ===== EDUCATION =====
    if any(k in t for k in [
        'teacher', 'professor', 'lecturer', 'educator', 'school'
    ]):
        return "Education"

    # ===== LEGAL =====
    if any(k in t for k in [
        'attorney', 'lawyer', 'legal', 'paralegal'
    ]):
        return "Legal"

    # ===== OPERATIONS / ADMIN =====
    if any(k in t for k in [
        'operations', 'admin', 'administrative', 'office', 'assistant',
        'secretary', 'coordinator'
    ]):
        return "Operations / Admin"

    # ===== MANAGEMENT / EXECUTIVE =====
    if any(k in t for k in [
        'manager', 'director', 'vp', 'chief', 'cfo', 'ceo', 'cto', 'lead', 'head'
    ]):
        return "Management / Leadership"

    # ===== SKILLED TRADES =====
    if any(k in t for k in [
        'mechanic', 'electrician', 'plumber', 'carpenter', 'welder', 'technician'
    ]):
        return "Skilled Trades"

    # ===== MISC =====
    return "Other"


In [47]:
final_df['Job_family'] = final_df['Job_title'].apply(map_job_family)


In [48]:
final_df['Job_family'].value_counts()


,count
Job_family,
Other,7731
Management / Leadership,4461
Operations / Admin,3377
Software Engineering,3144
Product / Project,1801
Data / AI,1380
Marketing,1080
Education,917
HR / Recruiting,754


# Normalizarion

Normalization means scaling numeric values so they fall within a smaller, more consistent range.



**Min–Max Normalization (0 to 1 scaling)**

Good for ML models like neural networks.

In [49]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

final_df['salary_minmax'] = scaler.fit_transform(final_df[['Salary']])


**Z-Score Normalization (Standard Scaling)**

Formula:


x′ =(𝑥−𝜇) / 𝜎

→ Centers data around mean = 0, std = 1

→ Useful when values have outliers (salary does!)

In [55]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

final_df['salary_zscore'] = scaler.fit_transform(final_df[['Salary']])


**Decimal Scaling Normalization**

Move decimal point until value is between −1 and 1.

Example:

Max salary = 500,000 → 6 digits → divide by 10^6


In [51]:
import numpy as np

max_salary = final_df['Salary'].abs().max()
j = len(str(int(max_salary)))  # number of digits

final_df['salary_decimal_scaled'] = final_df['Salary'] / (10**j)


| Method              | Best used when     | Good for salary?                  |
| ------------------- | ------------------ | --------------------------------- |
| **Min–Max**         | Want values in 0–1 | ❌ Salary has outliers → DANGEROUS |
| **Z-Score**         | Data has outliers  | ✅ BEST for salary                 |
| **Decimal scaling** | Simple compression | ❌ Not effective for skewed salary |
